In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from ultralytics import YOLO

In [ ]:
dataset_yolo = 'Peaple_Detection'
epoch_number = 200
weights_path = 'YoloV8/yolov8n.pt'
data_yaml_path = f'{dataset_yolo}/data.yaml'
saved_path = f'{dataset_yolo}/trained_model'
results_folder_name = f"{dataset_yolo}_normal"

model = YOLO(weights_path)
model.train(
    data=data_yaml_path,
    epochs=epoch_number,
    batch=32,
    device='cuda',
    project=saved_path,
    name=results_folder_name,
    patience=epoch_number,
    pretrained=True,
    lr0=0.001,
    lrf=0.001,
    dropout=0.2
)

In [ ]:
model_path = f"{results_folder_name}/weights/best.pt"
model = YOLO(model_path)
model.eval()

# transform = transforms.Compose([
#     transforms.Resize((img_size, img_size)),
#     transforms.ToTensor(),
# ])

In [ ]:
class CustomLoss(nn.Module):
    def __init__(self, alpha=0.8, beta=0.5, gamma=0.7):
        super(CustomLoss, self).__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.epsilon = 1e-6

    def forward(self, outputs, targets):
        cls_prob = outputs['cls_prob']  # [batch_size, num_anchors, num_classes]
        conf = outputs['conf']          # [batch_size, num_anchors]

        person_class_index = targets['person_class_index']

        person_cls_prob = cls_prob[:, :, person_class_index]  # [batch_size, num_anchors]

        person_mask = targets['person_mask'].float()  # [batch_size, num_anchors]
        background_mask = 1.0 - person_mask       

        L_bg = -torch.sum(
            (torch.log(person_cls_prob + self.epsilon) + torch.log(conf + self.epsilon)) * background_mask
        ) / (torch.sum(background_mask) + self.epsilon)

        L_person = torch.sum(
            (torch.log(1.0 - person_cls_prob + self.epsilon) + torch.log(1.0 - conf + self.epsilon)) * person_mask
        ) / (torch.sum(person_mask) + self.epsilon)

        L_bbox = -torch.mean(torch.log(conf + self.epsilon))
        loss = self.alpha * L_bg + self.beta * L_person + self.gamma * L_bbox

        return loss

In [ ]:
def pgd_attack(model, images, targets, epsilon, alpha, num_iter):
    images_adv = images.clone().detach().requires_grad_(True)
    loss_fn = CustomLoss()

    for i in range(num_iter):
        outputs = model(images_adv)
        loss = loss_fn(outputs, targets)
        loss.backward()

        grad_sign = images_adv.grad.sign()
        images_adv = images_adv + alpha * grad_sign

        eta = torch.clamp(images_adv - images, min=-epsilon, max=epsilon)
        images_adv = torch.clamp(images + eta, min=0, max=1).detach().requires_grad_(True)

    return images_adv